# MODELOS FINALES

In [1]:
import pandas as pd
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed
from tensorflow.keras.losses import mean_squared_error

In [2]:
completeRentalsDf = pd.read_csv('../data/completeData/rentals/completeRentalsDf_LOG1P.csv')
completeReturnsDf = pd.read_csv('../data/completeData/returns/completeReturnsDf_LOG1P.csv')

In [3]:
completeRentalsDf['Date'] = pd.to_datetime(completeRentalsDf['Date'])
completeRentalsDf.set_index('Date', inplace=True)
completeReturnsDf['Date'] = pd.to_datetime(completeReturnsDf['Date'])
completeReturnsDf.set_index('Date', inplace=True)

In [4]:
completeWeatherDf = pd.read_csv('../data/completeData/weather/completeWeatherDf.csv')

In [5]:
def getTrainValidationTestStationT24T48T168_ExogVariables(trainDays, validationDays, station, completeDf, completeWeatherDf):
    X_24 = np.delete(completeDf[[station]].values.reshape(-1, 24), [range(0, 6)], 0)
    X_48 = completeDf[[station]].values.reshape(-1, 24)
    X_48 = np.delete(X_48, [range(0, 5)], 0)
    X_48 = np.delete(X_48, len(X_48)-1, 0)
    X_168 = completeDf[[station]].values.reshape(-1, 24)
    X_168 = np.delete(X_168, [[len(X_168)-i for i in range(1, 7)]], 0)

    X_temp = np.delete(completeWeatherDf['temp'].values.reshape(-1, 24), [range(0, 7)], 0)
    X_humidity = np.delete(completeWeatherDf['humidity'].values.reshape(-1, 24), [range(0, 7)], 0)

    X = np.append( np.append( X_168, X_48, axis=1), X_24, axis = 1)
    X = np.delete(X, len(X)-1, 0)
    X = np.append( np.append( X_temp, X_humidity, axis=1), X, axis=1).reshape(-1,5, 24)

    X_transposed = np.empty((len(X), 24, 5))
    for i in range(0, len(X)):
        X_transposed[i] = X[i].transpose()

    X = X_transposed
    Y = np.delete(X_24, 0, 0)

    X_train, Y_train = X[0:trainDays], Y[0:trainDays]
    X_validation, Y_validation = X[trainDays:trainDays+validationDays], Y[trainDays:trainDays+validationDays]
    X_test , Y_test = X[trainDays+validationDays:X.shape[0]], Y[trainDays+validationDays:Y.shape[0]]

    return X_train, X_validation, X_test, Y_train, Y_validation, Y_test

def log1p_mse_rmse(Y_test, predictions):
    rmse = 0
    mse = 0
    for i in range(0, len(predictions)):
        mse += mean_squared_error(np.expm1(Y_test[i]), np.expm1(predictions[i]))

    mse /= len(predictions)
    rmse += math.sqrt(mse)
    print("MSE = {}, RMSE = {}".format(mse,rmse))

In [6]:
epochs = 10000
batch_size = 4096

In [7]:
stations = np.loadtxt("../data/completeData/stations/frequentedStations.txt")

### MODEL SERIALIZATION

In [8]:
X_train, X_validation, X_test, Y_train, Y_validation, Y_test = getTrainValidationTestStationT24T48T168_ExogVariables(4000, 102, '31201', completeRentalsDf, completeWeatherDf)
def getModel(X_train):
    model = Sequential()
    model.add(LSTM(X_train.shape[1], input_shape=(X_train.shape[1], 5), return_sequences=True, name='LSTM_1'))
    model.add(TimeDistributed(Dense(16, activation="relu", name="Dense_1")))
    model.add(Dense(8, activation='relu', name='Dense_2'))
    model.add(Dense(4, activation='relu', name='Dense_3'))
    model.add(Dense(2, activation='relu', name='Dense_4'))
    model.add(Dense(1, activation='relu', name='Dense_5'))
    model.compile(loss='mean_squared_error', optimizer='adamax', metrics=['mae'])
    return model

model = getModel(X_train)
with open("../data/modelsData/LSTM_serialization.json", "w") as json_file:
    json_file.write(model.to_json())

### RENTALS

In [9]:
for station in stations:
    station = str(int(station))
    print("Modelo alquileres estación {}".format(station))
    X_train, X_validation, X_test, Y_train, Y_validation, Y_test = getTrainValidationTestStationT24T48T168_ExogVariables(4000, 102, station, completeRentalsDf, completeWeatherDf)

    lossHistory = np.zeros((10000,), dtype=int)
    lossHistory[0], lossHistory[9999] = 0,0
    while format(lossHistory[0], '4f') == format(lossHistory[9999], '4f'):
        model = getModel(X_train)
        with tf.device('/GPU:0'):
            model.fit(X_train, Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1), batch_size=batch_size, epochs=epochs, verbose=0)
        model.save_weights("../data/modelsData/weights/rentals/rentals_"+ station +".h5")
        lossHistory = model.history.history['loss']
        print("Loss epoch nº1: {} --> Loss epoch nº10000: {}".format(format(lossHistory[0], '4f'), format(lossHistory[9999], '4f')))


Modelo alquileres estación 31201
Loss epoch nº1: 2.424701 --> Loss epoch nº10000: 2.424701
Loss epoch nº1: 1.967015 --> Loss epoch nº10000: 0.243657
Modelo alquileres estación 31200
Loss epoch nº1: 2.240768 --> Loss epoch nº10000: 0.260512
Modelo alquileres estación 31101
Loss epoch nº1: 1.945661 --> Loss epoch nº10000: 0.272801
Modelo alquileres estación 31229
Loss epoch nº1: 2.077129 --> Loss epoch nº10000: 0.267185
Modelo alquileres estación 31214
Loss epoch nº1: 1.823200 --> Loss epoch nº10000: 0.260741
Modelo alquileres estación 31623
Loss epoch nº1: 2.846476 --> Loss epoch nº10000: 0.269039
Modelo alquileres estación 31203
Loss epoch nº1: 1.448707 --> Loss epoch nº10000: 0.276690
Modelo alquileres estación 31104
Loss epoch nº1: 1.572671 --> Loss epoch nº10000: 1.572671
Loss epoch nº1: 1.036292 --> Loss epoch nº10000: 0.280798
Modelo alquileres estación 31600
Loss epoch nº1: 1.598236 --> Loss epoch nº10000: 1.598236
Loss epoch nº1: 1.598236 --> Loss epoch nº10000: 1.598236
Loss ep

### RETURNS

In [10]:
for station in stations:
    station = str(int(station))
    print("Modelo devoluciones estación {}".format(station))
    X_train, X_validation, X_test, Y_train, Y_validation, Y_test = getTrainValidationTestStationT24T48T168_ExogVariables(4000, 102, station, completeReturnsDf, completeWeatherDf)

    lossHistory = np.zeros((10000,), dtype=int)
    lossHistory[0], lossHistory[9999] = 0,0
    while format(lossHistory[0], '4f') == format(lossHistory[9999], '4f'):
        model = getModel(X_train)
        with tf.device('/GPU:0'):
            model.fit(X_train, Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1), batch_size=batch_size, epochs=epochs, verbose=0)
        model.save_weights("../data/modelsData/weights/returns/returns_"+ station +".h5")
        lossHistory = model.history.history['loss']
        print("Loss epoch nº1: {} --> Loss epoch nº10000: {}".format(format(lossHistory[0], '4f'), format(lossHistory[9999], '4f')))

Modelo devoluciones estación 31201
Loss epoch nº1: 2.609834 --> Loss epoch nº10000: 0.230541
Modelo devoluciones estación 31200
Loss epoch nº1: 3.023365 --> Loss epoch nº10000: 0.241363
Modelo devoluciones estación 31101
Loss epoch nº1: 2.082462 --> Loss epoch nº10000: 2.082462
Loss epoch nº1: 2.082462 --> Loss epoch nº10000: 2.082462
Loss epoch nº1: 2.082462 --> Loss epoch nº10000: 2.082462
Loss epoch nº1: 2.047203 --> Loss epoch nº10000: 0.250589
Modelo devoluciones estación 31229
Loss epoch nº1: 1.918375 --> Loss epoch nº10000: 0.235093
Modelo devoluciones estación 31214
Loss epoch nº1: 1.864711 --> Loss epoch nº10000: 1.864711
Loss epoch nº1: 1.863884 --> Loss epoch nº10000: 0.236491
Modelo devoluciones estación 31623
Loss epoch nº1: 3.027580 --> Loss epoch nº10000: 3.027579
Modelo devoluciones estación 31203
Loss epoch nº1: 1.677022 --> Loss epoch nº10000: 0.249074
Modelo devoluciones estación 31104
Loss epoch nº1: 1.329592 --> Loss epoch nº10000: 1.329592
Loss epoch nº1: 1.329592